In [1]:
import os
import requests

import pandas as pd

In [2]:
local = 'ko_KR'
startDate = '20000101'
endDate = '20230331'

# 개별종목 목록
def getStockDataFrame():
    url = f'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

    params = {
        'locale': local,
        'mktsel': 'ALL',
        'typeNo': 0,
        'searchText': '',
        'bld': 'dbms/comm/finder/finder_stkisu',
    }

    res = requests.post(url, data=params)

    df = pd.DataFrame(res.json()['block1'])
    return df

# get excel download code
def getExcelDownloadCode(stockName, fullCode, codeName):
    url = f'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'

    params = {
        'locale': local,
        'tboxisuCd_finder_stkisu0_1': stockName,
        'isuCd': fullCode,
        'isuCd2': '',
        'codeNmisuCd_finder_stkisu0_1': codeName,
        'param1isuCd_finder_stkisu0_1': 'ALL',
        'strtDd': startDate,
        'endDd': endDate,
        'adjStkPrc_check': 'Y',
        'adjStkPrc': 2,
        'share': 1,
        'money': 1,
        'csvxls_isNo': False,
        'name': 'fileDown',
        'url': 'dbms/MDC/STAT/standard/MDCSTAT01701'
    }

    res = requests.post(url, data=params)
    return res.text

# excel download
def downloadExcel(code, stockName):
    url = f'http://data.krx.co.kr/comm/fileDn/download_excel/download.cmd'

    params = {
        'code': code
    }
    
    res = requests.post(url, data=params)
    
    print(stockName + " : " + str(res))
    
    if not os.path.exists('stock_data_kr'):
        os.mkdir('stock_data_kr')
    
    file_name = 'stock_data_kr/' + stockName.replace('/', '_')
    if startDate != '':
        file_name += '_' + startDate
    if endDate != '':
        file_name += '_' + endDate
    file_name += '.xlsx'
    
    with open(file_name, 'wb') as f:
        f.write(res.content)

In [3]:
for i, row in getStockDataFrame().iterrows():
    stockName = row['short_code'] + '/' + row['codeName']
    fullCode = row['full_code']
    codeName = row['codeName']
    
    code = getExcelDownloadCode(stockName, fullCode, codeName)
    downloadExcel(code, stockName)

060310/3S : <Response [200]>
095570/AJ네트웍스 : <Response [200]>
006840/AK홀딩스 : <Response [200]>
054620/APS홀딩스 : <Response [200]>
265520/AP시스템 : <Response [200]>
211270/AP위성 : <Response [200]>
027410/BGF : <Response [200]>
282330/BGF리테일 : <Response [200]>
126600/BGF에코머티리얼즈 : <Response [200]>
138930/BNK금융지주 : <Response [200]>
001460/BYC : <Response [200]>
001465/BYC우 : <Response [200]>
013720/CBI : <Response [200]>
001040/CJ : <Response [200]>
079160/CJ CGV : <Response [200]>
035760/CJ ENM : <Response [200]>
311690/CJ 바이오사이언스 : <Response [200]>
00104K/CJ4우(전환) : <Response [200]>
000120/CJ대한통운 : <Response [200]>
011150/CJ씨푸드 : <Response [200]>
011155/CJ씨푸드1우 : <Response [200]>
001045/CJ우 : <Response [200]>
097950/CJ제일제당 : <Response [200]>
097955/CJ제일제당 우 : <Response [200]>
051500/CJ프레시웨이 : <Response [200]>
058820/CMG제약 : <Response [200]>
023460/CNH : <Response [200]>
056730/CNT85 : <Response [200]>
065770/CS : <Response [200]>
083660/CSA 코스믹 : <Response [200]>
000590/CS홀딩스 : <Response [200]

003570/SNT다이내믹스 : <Response [200]>
064960/SNT모티브 : <Response [200]>
100840/SNT에너지 : <Response [200]>
036530/SNT홀딩스 : <Response [200]>
005610/SPC삼립 : <Response [200]>
011810/STX : <Response [200]>
077970/STX엔진 : <Response [200]>
071970/STX중공업 : <Response [200]>
002820/SUN&L : <Response [200]>
289080/SV인베스트먼트 : <Response [200]>
084870/TBH글로벌 : <Response [200]>
002710/TCC스틸 : <Response [200]>
089230/THE E&M : <Response [200]>
161570/THE MIDONG : <Response [200]>
032540/TJ미디어 : <Response [200]>
022220/TKG애강 : <Response [200]>
069260/TKG휴켐스 : <Response [200]>
048770/TPC : <Response [200]>
246690/TS인베스트먼트 : <Response [200]>
317240/TS트릴리온 : <Response [200]>
002900/TYM : <Response [200]>
024070/WISCOM : <Response [200]>
057030/YBM넷 : <Response [200]>
037270/YG PLUS : <Response [200]>
040300/YTN : <Response [200]>
051390/YW : <Response [200]>
052220/iMBC : <Response [200]>
079940/가비아 : <Response [200]>
078890/가온미디어 : <Response [200]>
000500/가온전선 : <Response [200]>
399720/가온칩스 : <Response [200]>

442310/대신밸런스제14호스팩 : <Response [200]>
020180/대신정보통신 : <Response [200]>
003540/대신증권 : <Response [200]>
003547/대신증권2우B : <Response [200]>
003545/대신증권우 : <Response [200]>
045390/대아티아이 : <Response [200]>
009190/대양금속 : <Response [200]>
108380/대양전기공업 : <Response [200]>
006580/대양제지 : <Response [200]>
014160/대영포장 : <Response [200]>
047040/대우건설 : <Response [200]>
009320/대우부품 : <Response [200]>
042660/대우조선해양 : <Response [200]>
003090/대웅 : <Response [200]>
069620/대웅제약 : <Response [200]>
007680/대원 : <Response [200]>
000430/대원강업 : <Response [200]>
048910/대원미디어 : <Response [200]>
005710/대원산업 : <Response [200]>
006340/대원전선 : <Response [200]>
006345/대원전선우 : <Response [200]>
003220/대원제약 : <Response [200]>
024890/대원화성 : <Response [200]>
290380/대유 : <Response [200]>
002880/대유에이텍 : <Response [200]>
290120/대유에이피 : <Response [200]>
000300/대유플러스 : <Response [200]>
120240/대정화금 : <Response [200]>
003310/대주산업 : <Response [200]>
114920/대주이엔티 : <Response [200]>
078600/대주전자재료 : <Response [200]>
012800/대창 : <Respon

098120/마이크로컨텍솔 : <Response [200]>
147760/마이크로프랜드 : <Response [200]>
377480/마인즈랩 : <Response [200]>
038290/마크로젠 : <Response [200]>
001080/만호제강 : <Response [200]>
267980/매일유업 : <Response [200]>
005990/매일홀딩스 : <Response [200]>
093520/매커스 : <Response [200]>
377030/맥스트 : <Response [200]>
088980/맥쿼리인프라 : <Response [200]>
094800/맵스리얼티1 : <Response [200]>
100590/머큐리 : <Response [200]>
067280/멀티캠퍼스 : <Response [200]>
072870/메가스터디 : <Response [200]>
215200/메가스터디교육 : <Response [200]>
133750/메가엠디 : <Response [200]>
235980/메드팩토 : <Response [200]>
200580/메디쎄이 : <Response [200]>
041920/메디아나 : <Response [200]>
233250/메디안디노스틱 : <Response [200]>
014100/메디앙스 : <Response [200]>
236340/메디젠휴먼케어 : <Response [200]>
054180/메디콕스 : <Response [200]>
086900/메디톡스 : <Response [200]>
078160/메디포스트 : <Response [200]>
065650/메디프론 : <Response [200]>
138040/메리츠금융지주 : <Response [200]>
008560/메리츠증권 : <Response [200]>
408920/메쎄이상 : <Response [200]>
021880/메이슨캐피탈 : <Response [200]>
140410/메지온 : <Response [200]>
241770/메카로 : <

003720/삼영화학 : <Response [200]>
023000/삼원강재 : <Response [200]>
004380/삼익THK : <Response [200]>
002450/삼익악기 : <Response [200]>
032280/삼일 : <Response [200]>
002290/삼일기업공사 : <Response [200]>
004440/삼일씨엔에스 : <Response [200]>
000520/삼일제약 : <Response [200]>
009770/삼정펄프 : <Response [200]>
037460/삼지전자 : <Response [200]>
032750/삼진 : <Response [200]>
054090/삼진엘앤디 : <Response [200]>
005500/삼진제약 : <Response [200]>
000250/삼천당제약 : <Response [200]>
004690/삼천리 : <Response [200]>
024950/삼천리자전거 : <Response [200]>
038500/삼표시멘트 : <Response [200]>
017480/삼현철강 : <Response [200]>
010960/삼호개발 : <Response [200]>
046390/삼화네트웍스 : <Response [200]>
004450/삼화왕관 : <Response [200]>
009470/삼화전기 : <Response [200]>
011230/삼화전자 : <Response [200]>
001820/삼화콘덴서 : <Response [200]>
000390/삼화페인트 : <Response [200]>
027580/상보 : <Response [200]>
038540/상상인 : <Response [200]>
101000/상상인인더스트리 : <Response [200]>
415580/상상인제3호스팩 : <Response [200]>
001290/상상인증권 : <Response [200]>
041650/상신브레이크 : <Response [200]>
091580/상신이디피 : <Respon

...

380440/엔에이치스팩19호 : <Response [200]>
391060/엔에이치스팩20호 : <Response [200]>
396770/엔에이치스팩22호 : <Response [200]>
422040/엔에이치스팩23호 : <Response [200]>
437780/엔에이치스팩24호 : <Response [200]>
438580/엔에이치스팩25호 : <Response [200]>
439410/엔에이치스팩26호 : <Response [200]>
440820/엔에이치스팩27호 : <Response [200]>
450410/엔에이치스팩28호 : <Response [200]>
265740/엔에프씨 : <Response [200]>
354200/엔젠바이오 : <Response [200]>
419080/엔젯 : <Response [200]>
093510/엔지브이아이 : <Response [200]>
208860/엔지스테크널러지 : <Response [200]>
183490/엔지켐생명과학 : <Response [200]>
085310/엔케이 : <Response [200]>
182400/엔케이맥스 : <Response [200]>
348370/엔켐 : <Response [200]>
069410/엔텔스 : <Response [200]>
227950/엔투텍 : <Response [200]>
291230/엔피 : <Response [200]>
198080/엔피디 : <Response [200]>
048830/엔피케이 : <Response [200]>
096870/엘디티 : <Response [200]>
276240/엘리비젼 : <Response [200]>
900140/엘브이엠씨홀딩스 : <Response [200]>
376190/엘비루셈 : <Response [200]>
061970/엘비세미콘 : <Response [200]>
138690/엘아이에스 : <Response [200]>
290650/엘앤씨바이오 : <Response [200]>
066970/엘앤에프 : <Re

003465/유화증권우 : <Response [200]>
191410/육일씨엔에쓰 : <Response [200]>
372170/윤성에프앤씨 : <Response [200]>
008730/율촌화학 : <Response [200]>
072770/율호 : <Response [200]>
008250/이건산업 : <Response [200]>
039020/이건홀딩스 : <Response [200]>
025820/이구산업 : <Response [200]>
067920/이글루 : <Response [200]>
044960/이글벳 : <Response [200]>
303530/이노뎁 : <Response [200]>
296640/이노룰스 : <Response [200]>
302430/이노메트리 : <Response [200]>
279060/이노벡스 : <Response [200]>
214320/이노션 : <Response [200]>
056090/이노시스 : <Response [200]>
073490/이노와이어리스 : <Response [200]>
215790/이노인스트루먼트 : <Response [200]>
344860/이노진 : <Response [200]>
246960/이노테라피 : <Response [200]>
088390/이녹스 : <Response [200]>
272290/이녹스첨단소재 : <Response [200]>
053350/이니텍 : <Response [200]>
041520/이라이콤 : <Response [200]>
264850/이랜시스 : <Response [200]>
054210/이랜텍 : <Response [200]>
164060/이루다 : <Response [200]>
065440/이루온 : <Response [200]>
088260/이리츠코크렙 : <Response [200]>
139480/이마트 : <Response [200]>
115610/이미지스 : <Response [200]>
078020/이베스트투자증권 : <Response [200

233990/질경이 : <Response [200]>
085660/차바이오텍 : <Response [200]>
261780/차백신연구소 : <Response [200]>
009310/참엔지니어링 : <Response [200]>
094850/참좋은여행 : <Response [200]>
004650/창해에탄올 : <Response [200]>
278280/천보 : <Response [200]>
000650/천일고속 : <Response [200]>
140290/청광건설 : <Response [200]>
362320/청담글로벌 : <Response [200]>
012600/청호ICT : <Response [200]>
066360/체리부로 : <Response [200]>
033250/체시스 : <Response [200]>
047820/초록뱀미디어 : <Response [200]>
131100/초록뱀이앤엠 : <Response [200]>
052300/초록뱀컴퍼니 : <Response [200]>
118000/초록뱀헬스케어 : <Response [200]>
094360/칩스앤미디어 : <Response [200]>
016790/카나리아바이오 : <Response [200]>
016920/카스 : <Response [200]>
284620/카이노스메드 : <Response [200]>
446600/카이바이오텍 : <Response [200]>
035720/카카오 : <Response [200]>
293490/카카오게임즈 : <Response [200]>
323410/카카오뱅크 : <Response [200]>
377300/카카오페이 : <Response [200]>
042000/카페24 : <Response [200]>
006380/카프로 : <Response [200]>
317530/캐리소프트 : <Response [200]>
071850/캐스텍코리아 : <Response [200]>
050110/캠시스 : <Response [200]>
119860/커넥트웨이브 

217950/파마리서치바이오 : <Response [200]>
208340/파멥신 : <Response [200]>
005690/파미셀 : <Response [200]>
177830/파버나인 : <Response [200]>
037070/파세코 : <Response [200]>
150900/파수 : <Response [200]>
037030/파워넷 : <Response [200]>
047310/파워로직스 : <Response [200]>
266870/파워풀엑스 : <Response [200]>
368770/파이버프로 : <Response [200]>
170790/파이오링크 : <Response [200]>
049120/파인디앤씨 : <Response [200]>
038950/파인디지털 : <Response [200]>
441270/파인엠텍 : <Response [200]>
106240/파인테크닉스 : <Response [200]>
131760/파인텍 : <Response [200]>
065690/파커스 : <Response [200]>
140860/파크시스템스 : <Response [200]>
091700/파트론 : <Response [200]>
202960/판도라티비 : <Response [200]>
032800/판타지오 : <Response [200]>
318010/팜스빌 : <Response [200]>
036580/팜스코 : <Response [200]>
027710/팜스토리 : <Response [200]>
004720/팜젠사이언스 : <Response [200]>
225590/패션플랫폼 : <Response [200]>
054300/팬스타엔터프라이즈 : <Response [200]>
068050/팬엔터테인먼트 : <Response [200]>
028670/팬오션 : <Response [200]>
222110/팬젠 : <Response [200]>
010820/퍼스텍 : <Response [200]>
016800/퍼시스 : <Response [200]

079170/한창산업 : <Response [200]>
009460/한창제지 : <Response [200]>
372910/한컴라이프케어 : <Response [200]>
054920/한컴위드 : <Response [200]>
002680/한탑 : <Response [200]>
000880/한화 : <Response [200]>
00088K/한화3우B : <Response [200]>
452260/한화갤러리아 : <Response [200]>
45226K/한화갤러리아우 : <Response [200]>
451800/한화리츠 : <Response [200]>
088350/한화생명 : <Response [200]>
000370/한화손해보험 : <Response [200]>
009830/한화솔루션 : <Response [200]>
009835/한화솔루션우 : <Response [200]>
272210/한화시스템 : <Response [200]>
012450/한화에어로스페이스 : <Response [200]>
000885/한화우 : <Response [200]>
003530/한화투자증권 : <Response [200]>
003535/한화투자증권우 : <Response [200]>
386580/한화플러스제2호스팩 : <Response [200]>
430460/한화플러스제3호스팩 : <Response [200]>
195870/해성디에스 : <Response [200]>
034810/해성산업 : <Response [200]>
03481K/해성산업1우 : <Response [200]>
076610/해성옵틱스 : <Response [200]>
059270/해성티피씨 : <Response [200]>
101530/해태제과식품 : <Response [200]>
220180/핸디소프트 : <Response [200]>
143210/핸즈코퍼레이션 : <Response [200]>
900270/헝셩그룹 : <Response [200]>
214180/헥토이노베이션 : <Response 

In [4]:
base_path = 'stock_data_kr'

if not os.path.exists(base_path):
    os.mkdir(base_path)

if len(os.listdir(base_path)) == 0:
    print("no data")

for file_name in os.listdir(base_path):
    csv_base_path = base_path + "_csv/"
    
    if not os.path.exists(csv_base_path):
        os.mkdir(csv_base_path)
    
    pd.read_excel(os.path.join(base_path, file_name)).to_csv(os.path.join(csv_base_path, os.path.splitext(file_name)[0] + '.csv'), index=False)

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

...

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/

/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/khg/miniconda3/envs/stock/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
